# `Analysis of SYNTHEA data`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-dark')
 
from sqlalchemy import create_engine,text
user = 'root'
passwords = '7003890541'
host = 'localhost'
port = 3306
database = 'synthea'

engine = create_engine(f"mysql+pymysql://{user}:{passwords}@{host}:{port}/{database}")
def sql(query , engine = engine):
    return pd.read_sql(query,engine)


def select(table_name):
    return sql(f'select * from {table_name} limit 5')


def run(query):
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [2]:
select('encounters').head(1)

,id,start,stop,patient,organization,provider,payer,class,description,basic_cost,total_cost,payer_coverage,reason
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23 17:45:28,2010-01-23 18:10:28,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,ambulatory,encounter for symptom,129.16,129.16,54.16,acute bronchitis (disorder)


In [3]:
readmission_encounter_id = sql(
    '''
    with duplicate as (
    select first.id,date(first.start) as prev_date, date(date_add(first.start, interval 30 day)) as after_30_days,
    second.id as next_id, date(second.start) as next_date
    from encounters first
    join encounters second
    where first.patient = second.patient and
    second.start > first.start and
    second.start <= date_add(first.start,interval 30 day) )
    select distinct next_id from duplicate;
'''
)

In [4]:
# query = '''
#     create table readmission_data as
#     select * from encounters;
# '''
# run(query)

In [5]:
# run('''
#     alter table readmission_data
#     add column readmission tinyint(1) default 0;    
# ''')

In [6]:
# ids = "', '".join(readmission_encounter_id['next_id'])
# ids = f"'{ids}'"      # wrap with quotes

# query = f"""
# UPDATE readmission_data
# SET readmission = 1
# WHERE id IN ({ids});
# """

# run(query)

In [7]:
select('readmission_data').head(1)

,id,start,stop,patient,organization,provider,payer,class,description,basic_cost,total_cost,payer_coverage,reason,readmission
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23 17:45:28,2010-01-23 18:10:28,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,ambulatory,encounter for symptom,129.16,129.16,54.16,acute bronchitis (disorder),0


`MAIN DATASET IS CREATED FOR ANALYSIS`

`IMP TABLES FOR PROBLEM ->`

`MEDICATIONS`
`PATIENTS`
`CONDITIONS`
`PROCEDURES`
`CAREPLANS`

In [8]:
select('medications')

,start,stop,patient,payer,encounter,medicine_code,base_cost,payer_coverage,dispenses,total_cost,reason
0,2010-05-05 00:26:23,2011-04-30 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,677.08,0.0,12,8124.96,unknown
1,2011-04-30 00:26:23,2012-04-24 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,389221,624.09,0.0,12,7489.08,unknown
2,2012-04-24 00:26:23,2013-04-19 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7253a9f9-6f6d-429a-926a-7b1d424eae3f,748856,43.32,0.0,12,519.84,unknown
3,2011-05-13 12:58:08,2011-05-27 12:58:08,10339b10-3cd1-4ac3-ac13-ec26728cb592,d47b3510-2895-3b70-9897-342d681c769d,e1ab4933-07a1-49f0-b4bd-05500919061d,313782,8.14,0.0,1,8.14,acute bronchitis (disorder)
4,2011-12-08 15:02:18,2011-12-22 15:02:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,792fae81-a007-44b0-8221-46953737b089,562251,11.91,0.0,1,11.91,viral sinusitis (disorder)


In [22]:
query = '''
    select round(max(datediff(stop,start))) as medication_duration,
    encounter,
    count(distinct medicine_code) as medicins,
    sum(base_cost) as medicine_base_cost,
    sum(total_cost) as medicine_total_cost
    from medications
    group by encounter
'''

medications_agg = sql(query)

In [28]:
medications_agg.sample(1)

,medication_duration,encounter,medicins,medicine_base_cost,medicine_total_cost
17865,357.0,a79daf87-546e-48ae-84b6-d7374d7bf2aa,4,331.06,2068.06


In [29]:
select('patients').head(1)

,id,birthdate,marital,race,ethnicity,gender,city,state,county,expenses,coverage
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,m,white,hispanic,m,chicopee,massachusetts,hampden county,271227.08,1334.88


In [26]:
query = '''
    select id as patient_id,
    (timestampdiff(year,birthdate,"2020-12-31")) as patient_age,
    marital,race,ethnicity,gender
    from patients
'''

patient_agg = sql(query)

In [27]:
patient_agg.sample(1)

,patient_id,patient_age,marital,race,ethnicity,gender
569,cda36bac-8f55-4856-b586-140314f6cd89,40,m,white,nonhispanic,f


In [30]:
select('conditions').head(1)

,start,stop,patient,encounter,description
0,2001-05-01,None,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,chronic sinusitis (disorder)


In [15]:
query = '''
    select encounter, count(distinct description) as total_conditions from conditions
    group by encounter
'''
conditions_agg = sql(query)

In [31]:
conditions_agg.sample(1)

,encounter,total_conditions
3317,6ffcbab7-2e4a-4f36-a81b-f3bf420ff4fe,1


In [32]:
select('procedures').head(1)

,date,patient,encounter,description,base_cost,reason
0,2011-04-30t00:26:23z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,insertion of subcutaneous contraceptive,14896.56,unknow


In [33]:
query = '''
    select encounter, 
    count(distinct description) as total_procedures,
    count(distinct reason) as total_procedures_reasons,
    sum(base_cost) as total_procedure_cost
    from procedures
    group by encounter
'''
procedures_agg = sql(query)

In [34]:
procedures_agg.sample(1)

,encounter,total_procedures,total_procedures_reasons,total_procedure_cost
19312,eab460fb-70cc-4384-8120-79666de165aa,1,1,516.65


In [35]:
select('careplans').head(1)

,id,start,stop,patient,encounter,description,reason
0,d2500b8c-e830-433a-8b9d-368d30741520,2010-01-23,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,d0c40d10-8d87-447e-836e-99d26ad52ea5,respiratory therapy,acute bronchitis (disorder)


In [37]:
query = '''
        select 
        encounter,
        round(max(datediff(stop,start))) as careplan_duration,
        count(distinct description) as total_careplans,
        count(distinct reason) as total_careplan_reasons
        from careplans 
        group by encounter 
'''

careplans_agg = sql(query)

In [38]:
careplans_agg.sample(1)

,encounter,careplan_duration,total_careplans,total_careplan_reasons
205,0ed9beb4-7c3b-44ea-92fc-fa96146322f1,NaN,1,1


In [21]:
select('readmission_data')

,id,start,stop,patient,organization,provider,payer,class,description,basic_cost,total_cost,payer_coverage,reason,readmission
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23 17:45:28,2010-01-23 18:10:28,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,ambulatory,encounter for symptom,129.16,129.16,54.16,acute bronchitis (disorder),0
1,e88bc3a9-007c-405e-aabc-792a38f4aa2b,2012-01-23 17:45:28,2012-01-23 18:00:28,034e9e3b-2def-4559-bb2a-7850888ae060,772ee193-bb9f-30eb-9939-21e86c8e4da5,6f1d59a7-a5bd-3cf9-9671-5bad2f351c28,6e2f1a2d-27bd-3701-8d08-dae202c58632,wellness,general examination of patient (procedure),129.16,129.16,129.16,unknown,0
2,8f104aa7-4ca9-4473-885a-bba2437df588,2001-05-01 15:02:18,2001-05-01 15:17:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,5d4b9df1-93ae-3bc9-b680-03249990e558,af01a385-31d3-3c77-8fdb-2867fe88df2f,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,encounter for symptom,129.16,129.16,0.00,sinusitis (disorder),0
3,b85c339a-6076-43ed-b9d0-9cf013dec49d,2011-07-28 15:02:18,2011-07-28 15:17:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,3dc9bb2d-5d66-3e61-bf9a-e234c6433577,bb17e691-262b-3546-93d5-d88e7de93246,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,wellness,general examination of patient (procedure),129.16,129.16,0.00,unknown,0
4,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,2010-07-27 12:58:08,2010-07-27 13:28:08,10339b10-3cd1-4ac3-ac13-ec26728cb592,b03dba4f-892f-365c-bfd1-bfcfa7a98d5d,7ed6b84a-b847-3744-9d42-15c42297a0c2,d47b3510-2895-3b70-9897-342d681c769d,wellness,general examination of patient (procedure),129.16,129.16,129.16,unknown,0


In [44]:
query='''
    select id as encounter_id,
    start as encounter_start,
    stop as encounter_stopped,
    patient,organization,provider,class,readmission
    from readmission_data;
'''    
encounter_agg = sql(query)

`ALL THE DATA IS AGGREGATED AND READY TO MERGE`

In [45]:
patient_agg.sample(1)

,patient_id,patient_age,marital,race,ethnicity,gender
117,71e13815-55fb-4734-bcac-6079160d82a0,47,m,white,nonhispanic,f


In [46]:
medications_agg.sample(1)

,medication_duration,encounter,medicins,medicine_base_cost,medicine_total_cost
27056,0.0,fe1116b7-8c8f-4199-bf05-0c8e50f06bea,1,263.49,263.49


In [47]:
careplans_agg.sample(1)

,encounter,careplan_duration,total_careplans,total_careplan_reasons
218,0fab2427-2420-454d-af65-096d16aecb3f,28.0,1,1


In [48]:
conditions_agg.sample(1)

,encounter,total_conditions
6934,e6e1adcb-bced-4794-bdbf-69342b882584,1


In [63]:
encounter_agg.shape

(53346, 8)

In [64]:
data = (encounter_agg.merge(conditions_agg,left_on='encounter_id',right_on='encounter',how= 'left')
    .merge(careplans_agg,left_on='encounter_id',right_on='encounter',how= 'left')
    .merge(medications_agg,left_on='encounter_id',right_on='encounter',how= 'left')
    .merge(patient_agg,left_on='patient',right_on='patient_id',how= 'left')
)

In [65]:
data.head(1)

,encounter_id,encounter_start,encounter_stopped,patient,organization,provider,class,readmission,encounter_x,total_conditions,...,encounter,medicins,medicine_base_cost,medicine_total_cost,patient_id,patient_age,marital,race,ethnicity,gender
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23 17:45:28,2010-01-23 18:10:28,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,ambulatory,0,NaN,NaN,...,NaN,NaN,NaN,NaN,034e9e3b-2def-4559-bb2a-7850888ae060,37,m,white,nonhispanic,m


In [66]:
data.to_csv('data.csv',index = False)

In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53346 entries, 0 to 53345
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   encounter_id            53346 non-null  object        
 1   encounter_start         53346 non-null  datetime64[ns]
 2   encounter_stopped       53346 non-null  datetime64[ns]
 3   patient                 53346 non-null  object        
 4   organization            53346 non-null  object        
 5   provider                53346 non-null  object        
 6   class                   53346 non-null  object        
 7   readmission             53346 non-null  int64         
 8   encounter_x             7650 non-null   object        
 9   total_conditions        7650 non-null   float64       
 10  encounter_y             3472 non-null   object        
 11  careplan_duration       1951 non-null   float64       
 12  total_careplans         3472 non-null   float6